## Import Python module

In [6]:
import base64
from Crypto import Random
from Crypto.Cipher import AES
import secrets

In [7]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

In [8]:
import sqlite3

In [9]:
import datetime
import time
import pandas as pd
import os

#### * SQL문

In [10]:
select_all = "SELECT * FROM {}"
select_key = "SELECT * from KEY"
insert_webuser = "INSERT INTO webuser VALUES (?,?,?,?,?,?,?,?,?)"
drop_table = "DROP TABLE {}"
delete_webuser_by_idx = "DELETE FROM webuser WHERE idx=?"

In [11]:
# cur.execute("CREATE TABLE webuser(idx text, year int, month int, day int, time text, ip text, device text, pid text, url text)")
# cur.execute("CREATE TABLE KEY(ipkey text, devicekey text, pidkey text)")

In [10]:
# cur.execute(drop_table.format("webuser"))
# cur.execute(drop_table.format("KEY"))

In [ ]:
# 최초 1회만 생성 및 삽입
# ip_key = secrets.token_bytes(16)
# dev_key = secrets.token_bytes(16)
# pid_key = secrets.token_bytes(16)

# cur.execute("insert into KEY (ipkey, devicekey, pidkey) values (?,?,?)", (key['ip_key'] ,key['dev_key'], key['pid_key']))

#### * 암호문

In [11]:
class AES_cbcmode:
    
    def __init__(self,key):
        self.key = key
    
    def padding(self, raw_data):
        # 한국어의 경우 utf-8했을 때의 len이 달라지므로 encoding 과정 추가
        enc = raw_data.encode('utf-8')
        pad_num = AES.block_size - len(enc) % AES.block_size
        pad_chr = chr(pad_num)
        
        # encoding해서 최종 pad_data 반환
        return (raw_data + pad_num * pad_chr).encode('utf-8')
    
    def unpadding(self, pad_data):
        return pad_data[:-pad_data[-1]]
    
    def encrypt(self, raw_data):
        pad_data = self.padding(raw_data)
    
        # iv: initial vector
        # 두 번째 키와 같은 존재, 
        # 블록 단위로 암호/복호하면서 변경되고 그 값은 다음 블록 단위 암호/복호 시 사용
        iv = Random.new().read( AES.block_size )
        
        # CBC mode(Cipher Block Chaining)
        cipher = AES.new( self.key, AES.MODE_CBC, iv )
        return base64.b64encode(iv + cipher.encrypt(pad_data))
    
    def decrypt(self, enc_data):
        enc_decode = base64.b64decode(enc_data)
        iv = enc_decode[:AES.block_size]
        cipher = AES.new(self.key,AES.MODE_CBC, iv)
        return self.unpadding(cipher.decrypt(enc_decode[AES.block_size:]))

### 1. user DB 연결

In [12]:
conn = sqlite3.connect("user_db/user.db")
cur = conn.cursor()

### 2. Key 불러오기

In [13]:
key_df = pd.read_sql_query(select_key ,conn)
key_dict = dict(zip(key_df.columns, key_df.iloc[0]))

### 3. 사용자 정보 디바이스로부터 읽어오기 + 암호화 후 추가

#### 임시
***

In [14]:
path = "user_db/"
file_list = os.listdir(path)

In [15]:
user = file_list[0]

***

In [16]:
sel_num = 1

In [17]:
info=[]

# 시간
now = time.localtime()

for i in range(3):
    info.append(now[i])
    
info.append("%02d:%02d:%02d" %(now[3],now[4],now[5]))

# 사용자정보(from mobilephone)

# 1) ip & url
f = open(path+user+"/capture/{}.txt".format(sel_num), 'r')
ip = f.readline().strip('\n').rjust(64,'*')
enc_ip = AES_cbcmode(key_dict['ipkey']).encrypt(ip)
url = f.readline().strip('\n')
f.close

info.append(enc_ip)

# 2) device
f = open(path+user+"/device.txt", 'r')
device = f.readline().strip('\n').rjust(64,'*')
f.close()

enc_device = AES_cbcmode(key_dict['devicekey']).encrypt(device)
info.append(enc_device)

# 3) pid
pid = user[-10:].rjust(64,'*')
enc_pid = AES_cbcmode(key_dict['pidkey']).encrypt(pid)
info.append(enc_pid)

# 4) append url
info.append(url)


####  idx(워터마크 text) 생성
- text
- (사이트고유문자)+(월일)+(사용자고유식별문구6자리)+(숫자4자리)

EX) 임의로 지정  
- NAVER: NNN  
- 사용자고유식별문구: DLFM2B  
- 숫자: 0  

###### 아래의 두 정보는 사용자 디바이스의 cache에 저장
사용자고유식별문구: 사용자별로 부여됨(반드시 맨 뒷 글자는 문자)  
숫자: 년월일 기준으로 00시에 초기화

In [34]:
protect_url = {"comic.naver.com": "NAV","webtoon.daum.net":"DAU"}

In [38]:
#sites
sites = "III"
for i in protect_url.keys():
    if i in url:
        sites = protect_url[i]

#date(MMDD)
wm_date = "%02d%02d"%(now[1],now[2])

# cache_code
f = open(path+user+"/cache_code.txt", 'r')
cache_user = f.readline().strip('\n')
f.close()

#cache num
f = open(path+user+"/cache_num.txt", 'r')
cache_num = int(f.readline().strip('\n'))
f.close()

add_num = str(cache_num + 1)

f = open(path+user+"/cache_num.txt", 'w')
f.write(add_num)
f.close()

wm_txt = sites + wm_date + cache_user + "%04d" % cache_num

In [39]:
wm_txt

'DAU0614MGQOHR0000'

#### 삽입 후 DB 적용 + 종료

In [40]:
info.insert(0,wm_txt)

cur.execute(insert_webuser, tuple(info))

conn.commit()

cur.close()
conn.close()

### 3) 텍스트 이미지화

- 나눔스퀘어 사용
- font 크기: 10
- 워터마크 크기: (148,13)

In [305]:
font = ImageFont.truetype("font/NanumSquareEB.ttf",10)

orig_size = font.getsize(wm_txt)
pad_size = (148,13) #font = 10

background = (0,0,0)

image = Image.new('RGB', pad_size, background)

draw = ImageDraw.Draw(image)

point = ((pad_size[0]-orig_size[0])//2, (pad_size[1]-orig_size[1])//2)

draw.text(point, wm_txt, (255, 255, 255), font = font)

# 저장
image.save("wm/{}_img{}.png".format(user[:-11],cache_num+1))